In [4]:
#pip install -U "ray==2.9.1" "ray[defau`lt]" "ray[tune]" "ray[serve]" "dask[complete]"


In [5]:
#pip install -U boto3 pandas pyarrow fastparquet xlrd XlsxWriter openpyxl pyarrow fastparquet orjson ipython ipykernel ipywidgets nltk

In [4]:
import numpy as np
import pandas as pd
import ray
# from ray.util.dask import enable_dask_on_ray
# ray.init(ignore_reinit_error=True)
# enable_dask_on_ray()

In [2]:
import ray, time, glob
import dask.dataframe as dd
from ray.util.dask import ray_dask_get, enable_dask_on_ray, disable_dask_on_ray
import boto3

In [11]:
#df=pd.read_csv('s3://idq-cradle-output-bucket/sbiswaji/724acb42-2ca4-4aca-948d-1511b957b32e-0.tsv',sep='\t')
print(len(df.columns))
df.columns[df.columns.str.contains('.1')]
df.drop(columns=['gv1', 'asin.1', 'pl.1', 'gl.1', 'product_type.1', 'type.1', 'brand.1','dw_last_updated','dw_partition_id'],inplace=True)
print(len(df.columns))
df

31
24


,asin,month,year,gv,pl,gl,product_type,type,brand,selling_price,...,ytd_gv,caq_score,title_score,mac_overall_score,rac_overall_score,bp_score,pd_score,rbn_score,dw_last_updated,dw_partition_id
0,B09RV5KV2X,32,2023,871.0,Softlines,gl_apparel,BRA,ThreeP,SOIE,NaN,...,17985,0.667,0.0,0.885,1.000,1.000,1.0,1,2024-06-07 18:10:17.880955,20240607233430
1,B0C2YKVBQ4,36,2023,1.0,Softlines,gl_apparel,UNSTITCHED_GARMENT,ThreeP,TRENDMALLS,NaN,...,9032,0.667,0.0,0.818,0.786,1.000,1.0,0,2024-06-07 18:10:17.880955,20240607233430
2,B01J3OGVXS,51,2023,71.0,Softlines,gl_shoes,SHOES,RA,Vector X,NaN,...,3500,0.667,0.0,0.688,1.000,1.000,1.0,1,2024-06-07 18:10:17.880955,20240607233430
3,B096KT9VQ8,26,2022,288.0,Softlines,gl_shoes,SHOES,RA,Liberty,NaN,...,12120,0.667,0.0,0.771,1.000,1.000,1.0,1,2024-06-07 18:10:17.880955,20240607233430
4,B0B5MTFLL1,12,2023,37.0,Softlines,gl_beauty,SUNSCREEN,ThreeP,Amway,NaN,...,7145,0.625,0.0,0.875,0.750,0.875,1.0,1,2024-06-07 18:10:17.880955,20240607233430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6899233,B0CP65Q3MK,8,2024,96.0,Softlines,gl_watch,WATCH,ThreeP,Time Up,NaN,...,3676,0.667,0.0,0.875,1.000,1.000,1.0,1,2024-06-07 18:10:17.880955,20240607233430
6899234,B0CBHQYV3X,32,2023,17.0,Softlines,gl_apparel,PANTS,ThreeP,Generic,NaN,...,4062,0.667,0.0,0.781,0.813,1.000,1.0,1,2024-06-07 18:10:17.880955,20240607233430
6899235,B0C89Z2V41,32,2023,181.0,Softlines,gl_beauty,PERSONAL_FRAGRANCE,ThreeP,WILD STONE CODE,NaN,...,6012,0.667,0.0,0.917,0.917,1.000,1.0,1,2024-06-07 18:10:17.880955,20240607233430
6899236,B09XF8K4J8,52,2022,52.0,Softlines,gl_shoes,SANDAL,ThreeP,Marc Loire,NaN,...,71737,0.667,0.0,0.813,0.938,1.000,1.0,1,2024-06-07 18:10:17.880955,20240607233430


In [63]:
# df.drop(columns=['dw_last_updated','dw_partition_id'],inplace=True)
# df.head(10)
#df.sort_values(['asin','year','month'])

# df1=df[(df['month']==12) & (df['year']==2023)].sort_values(['asin','year','month'])
# df1
test_set=df[(df['month'].isin([1,2,3,4])) & (df['year']==2024) & (df['asin'].isin(df1['asin']))].sort_values(['asin','year','month'])

In [44]:
df.columns

Index(['asin', 'month', 'year', 'gv', 'pl', 'gl', 'product_type', 'type',
       'brand', 'selling_price', 'buy_days', 'new_asin_tier', 'run_dt',
       'wknum', 'ytd_gv', 'caq_score', 'title_score', 'mac_overall_score',
       'rac_overall_score', 'bp_score', 'pd_score', 'rbn_score'],
      dtype='object')

In [182]:
train_set=df[(df['year'].isin([2022,2023])) & (df['month'].isin([1,2,3,4,5,6])) & (df['asin'].isin(df1['asin']))].sort_values(['asin','year','month'])[['asin', 'month', 'year', 'gv', 'pl', 'gl', 'product_type', 'type', 'brand', 'selling_price', 'buy_days','run_dt']]
train_set[train_set['asin']=='0764333801']
train_set.reset_index(inplace=True,drop=True)
train_set['date']=train_set['year'].astype('str')+'-W'+train_set['month'].astype('str')

train_set['date']=train_set['date'].apply(lambda x: datetime.datetime.strptime(x+ '-1', "%Y-W%W-%w"))
train_set

,asin,month,year,gv,pl,gl,product_type,type,brand,selling_price,buy_days,run_dt,date
0,0764333801,1,2022,2333.00,Softlines,gl_watch,WATCH,RA,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-03
1,0764333801,2,2022,2681.00,Softlines,gl_watch,WATCH,RA,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-10
2,0764333801,3,2022,2305.00,Softlines,gl_watch,WATCH,RA,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-17
3,0764333801,4,2022,2392.00,Softlines,gl_watch,WATCH,RA,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-24
4,0764333801,5,2022,1964.00,Softlines,gl_watch,WATCH,RA,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
440310,B0BVBPNDM6,6,2023,1.00,Softlines,gl_apparel,APPAREL_HEAD_NECK_COVERING,ThreeP,The Headscarves,NaN,0,2024-06-01 00:00:00,2023-02-06
440311,B0BVBRC49D,6,2023,23.00,Softlines,gl_apparel,SHIRT,ThreeP,Indietoga,NaN,0,2024-06-01 00:00:00,2023-02-06
440312,B0BVBZ3SQR,6,2023,9.00,Softlines,gl_apparel,BRA_UNDERWEAR_SET,ThreeP,Glamyou,NaN,0,2024-06-01 00:00:00,2023-02-06
440313,B0BVBZKRNH,6,2023,3.00,Softlines,gl_apparel,DUPATTA,ThreeP,DIAMO,NaN,0,2024-06-01 00:00:00,2023-02-06


In [183]:
test_set=df[(df['month'].isin([1,2,3,4])) & (df['year']==2024) & (df['asin'].isin(df1['asin']))].sort_values(['asin','year','month'])[['asin', 'month', 'year', 'gv', 'pl', 'gl', 'product_type', 'type', 'brand', 'selling_price', 'buy_days','run_dt']]


In [70]:
import datetime
test_set.reset_index(inplace=True,drop=True)
test_set['date']=test_set['year'].astype('str')+'-W'+test_set['month'].astype('str')

test_set['date']=test_set['date'].apply(lambda x: datetime.datetime.strptime(x+ '-1', "%Y-W%W-%w"))
pd.to_datetime(test_set['date'])

0        2024-01-01
1        2024-01-08
2        2024-01-15
3        2024-01-22
4        2024-01-01
            ...    
208790   2024-01-22
208791   2024-01-01
208792   2024-01-08
208793   2024-01-15
208794   2024-01-22
Name: date, Length: 208795, dtype: datetime64[ns]

In [71]:
test_set['asin'].value_counts()

asin
0764333801    4
B0B39G486P    4
B0B394VHS6    4
B0B398YZ1S    4
B0B3993VM2    4
             ..
B07TSXGMXP    1
B0BPN1DBLJ    1
B0B5N1TBY4    1
B09J918B1M    1
B071XW7N9D    1
Name: count, Length: 52506, dtype: int64

In [78]:
train_set[train_set['selling_price'].isnull()!=True]
train_set

,asin,month,year,gv,pl,gl,product_type,type,brand,selling_price,buy_days
5185613,0764333801,1,2022,2333.0,Softlines,gl_watch,WATCH,RA,Schiffer Publishing,NaN,0
2159310,0764333801,2,2022,2681.0,Softlines,gl_watch,WATCH,RA,Schiffer Publishing,NaN,0
5698201,0764333801,3,2022,2305.0,Softlines,gl_watch,WATCH,RA,Schiffer Publishing,NaN,0
419482,0764333801,4,2022,2392.0,Softlines,gl_watch,WATCH,RA,Schiffer Publishing,NaN,0
2598341,0764333801,1,2023,4329.0,Softlines,gl_watch,WATCH,RA,Schiffer Publishing,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...
4595664,B0BTB19B27,4,2023,1.0,Softlines,gl_beauty,EYELID_COLOR,ThreeP,FOZZBY,NaN,2
170119,B0BTB1D4BD,4,2023,1.0,Softlines,gl_apparel,HAT,ThreeP,SELLORIA,NaN,2
5521956,B0BTB34RSR,4,2023,2.0,Softlines,gl_apparel,HAT,ThreeP,SELLORIA,NaN,2
4206444,B0BTB3CZ7V,4,2023,0.0,Softlines,gl_apparel,SKIRT,ThreeP,Generic,NaN,2


In [ ]:
df2.sort_values('date',ascending=False,inplace=True)
# Train=df2[pd.DatetimeIndex(df2['date']).year<2024].copy() ## Train data 2022-23
# Train

Train=Train[['date','gv']] ## Train data as per model input format
Train.columns=['ds','y']
Test=df2[pd.DatetimeIndex(df2['date']).year==2024] ## Test data 2024
Train

Test=Test[['date','gv']]## Train data as per model input format
Test.columns=['ds','y']
Test.sort_values('ds',inplace=True)
Test
m=Prophet(interval_width=0.95)
model=m.fit(Train)
forecast=m.predict(Test[['ds']])
forecast['asin']=i
re=pd.concat([re,forecast])

In [117]:
from category_encoders import TargetEncoder,OneHotEncoder
encode1 = OneHotEncoder(cols=['type'])
train_set=encode1.fit_transform(train_set)
#pd.get_dummies(train_set['type'], drop_first=True)
train_set

,asin,month,year,gv,pl,gl,product_type,type_1,type_2,brand,selling_price,buy_days,run_dt,date,Brand_encoded,PT_encoded,gv_log,ThreeP
0,0764333801,1,2022,2333.0,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-03,1531.671170,1091.185031,7.754910,False
1,0764333801,2,2022,2681.0,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-10,1531.671170,1091.185031,7.893945,False
2,0764333801,3,2022,2305.0,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-17,1531.671170,1091.185031,7.742836,False
3,0764333801,4,2022,2392.0,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-24,1531.671170,1091.185031,7.779885,False
4,0764333801,5,2022,1964.0,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-31,1531.671170,1091.185031,7.582738,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440310,B0BVBPNDM6,6,2023,1.0,Softlines,gl_apparel,APPAREL_HEAD_NECK_COVERING,0,1,The Headscarves,NaN,0,2024-06-01 00:00:00,2023-02-06,276.920869,646.888932,0.000000,True
440311,B0BVBRC49D,6,2023,23.0,Softlines,gl_apparel,SHIRT,0,1,Indietoga,NaN,0,2024-06-01 00:00:00,2023-02-06,135.181848,333.237381,3.135494,True
440312,B0BVBZ3SQR,6,2023,9.0,Softlines,gl_apparel,BRA_UNDERWEAR_SET,0,1,Glamyou,NaN,0,2024-06-01 00:00:00,2023-02-06,569.415288,604.485600,2.197225,True
440313,B0BVBZKRNH,6,2023,3.0,Softlines,gl_apparel,DUPATTA,0,1,DIAMO,NaN,0,2024-06-01 00:00:00,2023-02-06,440.432986,494.202068,1.098612,True


In [184]:
# Pre processing
import numpy as np
from category_encoders import TargetEncoder,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
encoder = TargetEncoder()
train_set['Brand_encoded'] = encoder.fit_transform(train_set[['brand']], train_set['gv'])
train_set['PT_encoded']=encoder.fit_transform(train_set[['product_type']], train_set['gv'])
train_set['gv'] = train_set['gv'].replace(0, 1e-6)
train_set['gv'] = train_set['gv'].clip(lower=1e-6)
scaler=MinMaxscaler()
train_set['gv_log']=np.log(train_set['gv'])

encode1 = OneHotEncoder(cols=['type'])
train_set=encode1.fit_transform(train_set)
train_set=train_set[train_set['asin'].isin(list(ab[ab['count']>2].index))]
train_set

,asin,month,year,gv,pl,gl,product_type,type_1,type_2,brand,selling_price,buy_days,run_dt,date,Brand_encoded,PT_encoded,gv_log
0,0764333801,1,2022,2333.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-03,1531.67,1091.19,7.75
1,0764333801,2,2022,2681.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-10,1531.67,1091.19,7.89
2,0764333801,3,2022,2305.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-17,1531.67,1091.19,7.74
3,0764333801,4,2022,2392.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-24,1531.67,1091.19,7.78
4,0764333801,5,2022,1964.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,2022-01-31,1531.67,1091.19,7.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439131,B0BTB3CZ7V,5,2023,0.00,Softlines,gl_apparel,SKIRT,0,1,Generic,NaN,7,2024-06-01 00:00:00,2023-01-30,405.75,293.27,-13.82
439132,B0BTB3CZ7V,6,2023,0.00,Softlines,gl_apparel,SKIRT,0,1,Generic,NaN,7,2024-06-01 00:00:00,2023-02-06,405.75,293.27,-13.82
439133,B0BTB4LB2V,4,2023,0.00,Softlines,gl_watch,WATCH_BAND,0,1,Meyaar,NaN,2,2024-06-01 00:00:00,2023-01-23,957.88,1174.29,-13.82
439134,B0BTB4LB2V,5,2023,9.00,Softlines,gl_watch,WATCH_BAND,0,1,Meyaar,NaN,7,2024-06-01 00:00:00,2023-01-30,957.88,1174.29,2.20


In [146]:
ab=pd.DataFrame(train_set['asin'].value_counts())


In [185]:
encoder = TargetEncoder()
test_set['Brand_encoded'] = encoder.fit_transform(test_set[['brand']], test_set['gv'])
test_set['PT_encoded']=encoder.fit_transform(test_set[['product_type']], test_set['gv'])
test_set['gv'] = test_set['gv'].replace(0, 1e-6)
test_set['gv'] = test_set['gv'].clip(lower=1e-6)
test_set['gv_log']=np.log(test_set['gv'])
test_set.reset_index(inplace=True,drop=True)
test_set['date']=test_set['year'].astype('str')+'-W'+test_set['month'].astype('str')
test_set['date']=test_set['date'].apply(lambda x: datetime.datetime.strptime(x+ '-1', "%Y-W%W-%w"))
encode1 = OneHotEncoder(cols=['type'])
test_set=encode1.fit_transform(test_set)
test_set=test_set[test_set['asin'].isin(list(ab[ab['count']>2].index))]
test_set

,asin,month,year,gv,pl,gl,product_type,type_1,type_2,brand,selling_price,buy_days,run_dt,Brand_encoded,PT_encoded,gv_log,date
0,0764333801,1,2024,3614.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,1327.55,978.72,8.19,2024-01-01
1,0764333801,2,2024,4467.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,1327.55,978.72,8.40,2024-01-08
2,0764333801,3,2024,4487.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,1327.55,978.72,8.41,2024-01-15
3,0764333801,4,2024,4326.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,1327.55,978.72,8.37,2024-01-22
4,2596343959,1,2024,177.00,Softlines,gl_shoes,SHOES,0,1,ACME,NaN,0,2024-06-01 00:00:00,668.62,893.90,5.18,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191849,B0BTB3CZ7V,4,2024,178.00,Softlines,gl_apparel,SKIRT,0,1,Generic,NaN,0,2024-06-01 00:00:00,366.86,298.39,5.18,2024-01-22
191850,B0BTB4LB2V,1,2024,247.00,Softlines,gl_watch,WATCH_BAND,0,1,Meyaar,NaN,0,2024-06-01 00:00:00,408.01,833.86,5.51,2024-01-01
191851,B0BTB4LB2V,2,2024,425.00,Softlines,gl_watch,WATCH_BAND,0,1,Meyaar,NaN,0,2024-06-01 00:00:00,408.01,833.86,6.05,2024-01-08
191852,B0BTB4LB2V,3,2024,647.00,Softlines,gl_watch,WATCH_BAND,0,1,Meyaar,NaN,0,2024-06-01 00:00:00,408.01,833.86,6.47,2024-01-15


In [101]:
from prophet import Prophet
m=Prophet()
#train_set=train_set[['date','gv']] ## Train data as per model input format
train_set.columns=['ds','y']
m.fit(train_set[['ds','y']][0])    
m.make_future_dataframe(periods=12, freq='W')

17:51:34 - cmdstanpy - INFO - Chain [1] start processing
17:51:46 - cmdstanpy - INFO - Chain [1] done processing


,ds
0,2022-01-03
1,2022-01-10
2,2022-01-17
3,2022-01-24
4,2022-01-31
5,2022-02-07
6,2023-01-02
7,2023-01-09
8,2023-01-16
9,2023-01-23


In [186]:
import warnings
warnings.filterwarnings("ignore")
re=pd.DataFrame()
aa=train_set['asin'].unique()
for i in aa[0:200]:
    df2=train_set[train_set['asin']==i].copy()
    df2.sort_values('date',ascending=False,inplace=True)
    #Train=df2[pd.DatetimeIndex(df2['date']).year<2024].copy() ## Train data 2022-23
    

    #df2=df2[['date','gv_log']] ## Train data as per model input format
    df2.rename(columns={'date': 'ds', 'gv': 'y'},inplace=True)
    #Test=df2[pd.DatetimeIndex(df2['date']).year==2024] ## Test data 2024
    
    model=Prophet(interval_width=0.95,yearly_seasonality=True)
    
    # model.add_regressor('Brand_encoded')
    # model.add_regressor('PT_encoded')
    # model.add_regressor('type_1')
    # model.add_country_holidays(country_name='IND')
    model.fit(df2)
    df3=test_set[test_set['asin']==i][['date','Brand_encoded','PT_encoded','type_1']]
    df3.rename(columns={'date': 'ds'},inplace=True)
    forecast=model.predict(df3[['ds']])
    forecast['asin']=i
    re=pd.concat([re,forecast])
print('done')

19:11:20 - cmdstanpy - INFO - Chain [1] start processing
19:11:20 - cmdstanpy - INFO - Chain [1] done processing
19:11:20 - cmdstanpy - INFO - Chain [1] start processing
19:11:20 - cmdstanpy - INFO - Chain [1] done processing
19:11:20 - cmdstanpy - INFO - Chain [1] start processing
19:11:20 - cmdstanpy - INFO - Chain [1] done processing
19:11:21 - cmdstanpy - INFO - Chain [1] start processing
19:11:22 - cmdstanpy - INFO - Chain [1] done processing
19:11:22 - cmdstanpy - INFO - Chain [1] start processing
19:11:22 - cmdstanpy - INFO - Chain [1] done processing
19:11:22 - cmdstanpy - INFO - Chain [1] start processing
19:11:22 - cmdstanpy - INFO - Chain [1] done processing
19:11:23 - cmdstanpy - INFO - Chain [1] start processing
19:11:23 - cmdstanpy - INFO - Chain [1] done processing
19:11:23 - cmdstanpy - INFO - Chain [1] start processing
19:11:23 - cmdstanpy - INFO - Chain [1] done processing
19:11:23 - cmdstanpy - INFO - Chain [1] start processing
19:11:23 - cmdstanpy - INFO - Chain [1]

done


In [187]:
re

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,asin
0,2024-01-01,2346.17,5700.28,7717.47,2346.17,2346.17,4328.51,4328.51,4328.51,4328.51,4328.51,4328.51,0.00,0.00,0.00,6674.68,0764333801
1,2024-01-08,2395.48,6362.46,8360.23,2395.48,2395.48,4951.56,4951.56,4951.56,4951.56,4951.56,4951.56,0.00,0.00,0.00,7347.04,0764333801
2,2024-01-15,2444.79,5992.22,8095.04,2444.79,2444.79,4575.30,4575.30,4575.30,4575.30,4575.30,4575.30,0.00,0.00,0.00,7020.09,0764333801
3,2024-01-22,2494.10,6114.48,8238.62,2494.10,2494.10,4720.73,4720.73,4720.73,4720.73,4720.73,4720.73,0.00,0.00,0.00,7214.83,0764333801
0,2024-01-01,434.32,295.25,328.83,434.32,434.32,-122.82,-122.82,-122.82,-122.82,-122.82,-122.82,0.00,0.00,0.00,311.50,2596343959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2024-01-22,144.27,309.45,440.40,144.27,144.27,231.11,231.11,231.11,231.11,231.11,231.11,0.00,0.00,0.00,375.37,B003RF82RI
0,2024-01-01,-107.36,-52.55,-17.44,-107.36,-107.36,73.03,73.03,73.03,73.03,73.03,73.03,0.00,0.00,0.00,-34.33,B003RYQ3OS
1,2024-01-08,-108.56,-94.14,-59.40,-108.56,-108.56,32.21,32.21,32.21,32.21,32.21,32.21,0.00,0.00,0.00,-76.36,B003RYQ3OS
2,2024-01-15,-109.76,-64.86,-31.10,-109.76,-109.76,61.81,61.81,61.81,61.81,61.81,61.81,0.00,0.00,0.00,-47.95,B003RYQ3OS


In [175]:
test_set

,asin,month,year,gv,pl,gl,product_type,type_1,type_2,brand,selling_price,buy_days,run_dt,Brand_encoded,PT_encoded,date
0,0764333801,1,2024,3614.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,1327.55,978.72,2024-01-01
1,0764333801,2,2024,4467.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,1327.55,978.72,2024-01-08
2,0764333801,3,2024,4487.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,1327.55,978.72,2024-01-15
3,0764333801,4,2024,4326.00,Softlines,gl_watch,WATCH,1,0,Schiffer Publishing,NaN,0,2024-06-01 00:00:00,1327.55,978.72,2024-01-22
4,2596343959,1,2024,177.00,Softlines,gl_shoes,SHOES,0,1,ACME,NaN,0,2024-06-01 00:00:00,668.62,893.90,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191849,B0BTB3CZ7V,4,2024,178.00,Softlines,gl_apparel,SKIRT,0,1,Generic,NaN,0,2024-06-01 00:00:00,366.86,298.39,2024-01-22
191850,B0BTB4LB2V,1,2024,247.00,Softlines,gl_watch,WATCH_BAND,0,1,Meyaar,NaN,0,2024-06-01 00:00:00,408.01,833.86,2024-01-01
191851,B0BTB4LB2V,2,2024,425.00,Softlines,gl_watch,WATCH_BAND,0,1,Meyaar,NaN,0,2024-06-01 00:00:00,408.01,833.86,2024-01-08
191852,B0BTB4LB2V,3,2024,647.00,Softlines,gl_watch,WATCH_BAND,0,1,Meyaar,NaN,0,2024-06-01 00:00:00,408.01,833.86,2024-01-15


In [188]:
re1=re[['asin','yhat','ds']]
#re1['reversed'] = np.exp(re1['yhat']).astype('float')
pd.options.display.float_format = '{:.2f}'.format
apd.merge(re1,test_set[['asin','gv','gv_log','date']],on='asin',how='left')

,asin,yhat,ds,gv,gv_log,date
0,0764333801,6674.68,2024-01-01,3614.00,8.19,2024-01-01
1,0764333801,6674.68,2024-01-01,4467.00,8.40,2024-01-08
2,0764333801,6674.68,2024-01-01,4487.00,8.41,2024-01-15
3,0764333801,6674.68,2024-01-01,4326.00,8.37,2024-01-22
4,0764333801,7347.04,2024-01-08,3614.00,8.19,2024-01-01
...,...,...,...,...,...,...
3188,B003RYQ3OS,-47.95,2024-01-15,245.00,5.50,2024-01-22
3189,B003RYQ3OS,-46.15,2024-01-22,182.00,5.20,2024-01-01
3190,B003RYQ3OS,-46.15,2024-01-22,154.00,5.04,2024-01-08
3191,B003RYQ3OS,-46.15,2024-01-22,297.00,5.69,2024-01-15


In [ ]:
Test=Test[['date','gv']]## Train data as per model input format
    Test.columns=['ds','y']
    Test.sort_values('ds',inplace=True)
    Test

,asin,month,year,gv,pl,gl,product_type,type_1,type_2,brand,selling_price,buy_days,run_dt,date,Brand_encoded,PT_encoded,gv_log
453,B000EP6HPI,3,2023,1.0,Softlines,gl_shoes,SHOES,0,1,Sperry Top-Sider,NaN,0,2024-06-01 00:00:00,2023-01-16,575.539662,623.048187,0.0
